In [ ]:
import pandas as pd
import numpy as np

In [ ]:
training_csv = pd.read_csv("SBIC.v2.agg.trn.csv", index_col=0)

In [ ]:
training_csv

# Manipulating the CSV file so that we only use the columns specified by Maarten
- post [CLS] offensiveYN [GRP] group [STE] implications [END]

In [ ]:
training_csv = training_csv.drop(["dataSource", "offensiveYN", "sexYN", "intentYN", "targetCategory"], axis=1, index=None)
training_csv.columns = ["post", "group", "implications", "whoTarget", "offensiveYN"]
training_csv

In [ ]:
training_csv

In [ ]:
#training_csv["group"] = training_csv["group"].apply(lambda label: "NaN" if label=='[]' else label)
training_csv["offensiveYN"] = training_csv["offensiveYN"].apply(lambda label: "[OffN]" if label==1 else "[OffY]")

In [ ]:
print(training_csv["offensiveYN"])

In [ ]:
training_csv["post"] = training_csv["post"].apply(lambda label: "[BOS]" + label + "[CLS]")

In [ ]:
training_csv.dtypes

In [ ]:
def createNewLine(i, group):
    copy_df = training_csv.copy(deep=True)
    copy_df.append(copy_df.loc[[i]])
    copy_df.loc[[len(training_csv)-1]]["implications"] = group[len(group)-1]
    del group[-1]
    copy_df.loc[i,"implications"] = str(group)
    return copy_df

In [ ]:
import ast
#one implication per line
# if > 1 implication, copy entire line, put on end, change group
list_implications = []
list_group = []
for i in training_csv.index:
    list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    if (len(list_val) > 1):
        while (len(list_val) > 1):
            training_csv = createNewLine(i, list_val)
            list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    list_group.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["group"].values[0]))))

In [ ]:
#training_csv["implications"] = list_implications
training_csv["group"] = list_group

In [ ]:
training_csv
#do final pass over the implications and get rid of list format
for i in training_csv.index:
    list_implications.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["implications"].values[0]))))

In [ ]:
training_csv["implications"] = list_implications
training_csv

In [ ]:
len(training_csv["implications"][0])

In [ ]:
final_list = []
for i in training_csv.index:
    if (training_csv.loc[[i]]["whoTarget"].values[0] == 0.0 and training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffY]"):
        final_list.append("[IND] [END]")
    elif (training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffN]"):
        final_list.append(" [END]")
    else:
        final_list.append("[GRP] " + str(training_csv.loc[[i]]["group"].values[0]))# + " [STE] " + str(training_csv.loc[[i]]["implications"].values[0]) + " [END]")

In [ ]:
training_csv["implications"] = training_csv["implications"].apply(lambda label: "[STE] " + label + " [END]" if len(label)>0 else label)

In [ ]:
training_csv["group"] = final_list
training_csv = training_csv.drop(["whoTarget"], axis=1)

[BOS] I hate all gay cause they are bad [CLS] [OffY] [GRP] gay men [STE] implies that gays are bad [END] \
[BOS] You're a moron [CLS] [OffY] [IND] [END] \
[BOS] I cannot stand women sport announcers [CLS] [OffY] [GRP] Women [STE] Women aren't knowledgeable about sports [END]

post [CLS] offensiveYN [GRP] group [STE] implications [END]

# scramble method

In [ ]:
final_scramble_output = []
final_scramble_input = []

In [ ]:
training_csv.loc[[2]].values.tolist()

### When scrambling, have the same posts multiple times with different inputs outputs scrambled

In [ ]:
len(training_csv)

In [ ]:
import random 
for i in range(0, len(training_csv.index)):
    # create 4 versions of each post in scrambled order (more data)
    for i in range(0,4):
        # create a list of all things to append to either input or output in the dataframe, append whatevers left to output
        toAppend = list(filter(lambda item: item.strip(), training_csv.loc[[i]].values.tolist()[0]))
        # randomly decide how much is appended to the output and how much for the input (at least one thing has to be in the input/output
        appendNum = random.randint(1, len(toAppend)-1)
        # loop by appendnum
        # new list er input and output, make into a sentence later
        listApp = []
        for i in range(0, appendNum) :
            # randomly decide what to append to the output and what to append to the input
            i = random.randint(0, len(toAppend)-1)
            value = toAppend.pop(i)
            listApp.append(value)
        final_scramble_input.append(listApp)
        # whatever is left goes to output
        final_scramble_output.append(toAppend)

In [ ]:
len(final_scramble_input) == 4 * len(training_csv)

In [ ]:
final_scramble_input[0]
final_scramble_output[0]

In [ ]:
# list comprehension - make each entry into one string
string_scramble_input = ["".join(f).strip() for f in final_scramble_input]
string_scramble_output = ["".join(f).strip() for f in final_scramble_output]

In [ ]:
string_scramble_input[0]

In [ ]:
final_df = pd.DataFrame()

In [ ]:
final_df["input"] = string_scramble_input
final_df["output"] = string_scramble_output

In [ ]:
string_scramble_input[0:5]

In [ ]:
string_scramble_output[0:5]

In [ ]:
final_df.to_csv("SBIC.dev.scrambled.csv", index=False)

In [ ]:
pd.read_csv("")

In [ ]:
# from https://homes.cs.washington.edu/~msap/pdfs/sap2020socialbiasframes.pdf and 
# https://arxiv.org/pdf/2109.05322v1.pdf
sample_input = []
samples = ["Kendrick just dropped the coldest video of all fucking time", "What a fucking stupid bitch shut the fuck up already",
           "need to fuck sum bad , I got some pressure built up :P", "I take a bitch home give her long dick", 
           "You annoyed me every day for months you’re a fucking moron", 
           "My problem with Korean artists: I dont know how to pronounce your name I can’t hanggul to save my life",
           "I’ll continue this thread till non-lifting fags stop lifting like idiots at the gym... Which is never.",
           "I thought drugs were the only things black people could shoot up Boy was I wrong",
           "white revolution is the only solution",
           "yes you are fine in a white majority country. how is mexico doing these days? people come here because you couldn’t build it.",
           "black people only seem to thrive in white countries. how strange", 
           "soon white people will not be a majority anywhere. you believe",
           "muslims are always wanting to kill someone! its their duty to kill the infidel.",
           "oh how i wish a man with a gun was there! wake up white people! they hate us"
        ]
for sample in samples:
    sample_input.append("[BOS] " + sample + " [CLS] ")

In [ ]:
sample_input

In [ ]:
sample_df = pd.DataFrame()
sample_df["input"] = sample_input


In [ ]:
sample_df.to_csv("samples.csv", index=False)

# Accuracy scores

- draw conclusions about what needs to happen, what are pitfalls/missing, more data needed? Which of the categories from latent hate they are (best guess, pick samples to show them)
- look at stereotypes, and guess what the model may need
- create demo to help with playing with it
- grab data that has already been released
- dynahate (subtle hatred)
- retrain group and append group to input and only guess stereotype
- input and output is whole string, take fraction as input and remain as output, train model on all forms of input (when using at inference time, it generates what you want)
- scramble method (try both models, see what happens)
- take examples from social boas, implicit hate, dynsahate, see what the model generates/compare to actual data, derive insights from this
- paper that maarten shared (they - how would model pick up on it)
- read papers in chat

In [ ]:
from sklearn.metrics import f1_score
import re

In [ ]:
y_test = pd.read_csv("SBIC.dev.1.csv")["output"]
#y_test = y_test.drop(["dataSource", "offensiveYN", "sexYN", "intentYN", "targetCategory"], axis=1, index=None)
#y_test.columns = ["post", "group", "implications", "whoTarget", "offensiveYN"]
#y_pred = pd.read_csv("predictions_scrambled.csv", index_col=0)["Generated Text"].values #TODO: fix path if necessary

In [ ]:
y_test

In [ ]:
# use reg expressions to split into offy, group, etc. - calculate accuracy on each
# tokens = ['[BOS]', '[CLS]', '[OFFY]', '[OFFN]', '[IND]', '[STE]', '[END]', '[GRP]']
res = []
# split on parenthesis
for i in range(0, len(y_test)):
    res.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                    y_test[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                    .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                    .split(']')])))
   # res.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in y_pred[i].replace("<pad>", '').replace("[", '').replace("END", '').replace("STE", '').replace("GRP", '').split(']')])))

In [ ]:
    #TODO: add all characters to re split re.split("[]+")
# binary acc for offn/y, ignore group/implication if OffN
res = pd.DataFrame(res)
res[0] = res[0].apply(lambda label: 1 if label=="OffN" else 0)

In [ ]:
res

In [ ]:
sum_int = sum(res[0].values == y_test["offensiveYN"])

# Accuracy of OFFy vs OFFn

In [ ]:
sum_int/len(res[0].values == y_test["offensiveYN"])

# F1 Score of OFFy vs OFFn

In [ ]:
f1_score(y_test["offensiveYN"].values, res[0].values)

In [ ]:
res[1]

# Qualitative Analysis - how does the models groupings relate to the original (maybe use contains?)

In [ ]:
# does it match any of the groups from the original 
# ngram - BLEU score nltk package to use 
# exact match and bleu score
match = []
for i in range(0,len(res)):
    if (list(y_test["offensiveYN"].values)[i] == 0):
        res_val = str(list(res[1].values)[i])
        y_test_val = list(y_test["group"].values)
        match.append(any(res_val in s for s in y_test_val))

In [ ]:
sum(match) / len(match)

# New Scrambler Method as Per last meeting
- need to overrepresent offensive samples and have more structure with the tokens so the model learns
- create each kind of scramble for each input
#### Most important:
1) generate stereotype given just post
2) generate stereotype given post, group, and offensiveness
3) generate post given group, sterotype, and offensiveness

Making three kinds of scrambles:
1) regular, with new tokens
2) full scramble, with new tokens
3) important scramble, with new tokens

In [ ]:
training_csv = pd.read_csv("SBIC.v2.agg.dev.csv", index_col=0)

In [ ]:
training_csv

In [ ]:
training_csv = training_csv.drop(["dataSource", "offensiveYN", "sexYN", "intentYN", "targetCategory"], axis=1, index=None)
training_csv.columns = ["post", "group", "implications", "whoTarget", "offensiveYN"]
training_csv

In [ ]:
#training_csv["group"] = training_csv["group"].apply(lambda label: "NaN" if label=='[]' else label)
training_csv["offensiveYN"] = training_csv["offensiveYN"].apply(lambda label: "[OffN]" if label==1 else "[OffY]")

In [ ]:
def createNewLine(i, group):
    copy_df = training_csv.copy(deep=True)
    copy_df.append(copy_df.loc[[i]])
    copy_df.loc[[len(training_csv)-1]]["implications"] = group[len(group)-1]
    del group[-1]
    copy_df.loc[i,"implications"] = str(group)
    return copy_df

In [ ]:
import ast
#one implication per line
# if > 1 implication, copy entire line, put on end, change group
list_implications = []
list_group = []
for i in training_csv.index:
    list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    if (len(list_val) > 1):
        while (len(list_val) > 1):
            training_csv = createNewLine(i, list_val)
            list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    list_group.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["group"].values[0]))))

In [ ]:
#training_csv["implications"] = list_implications
training_csv["group"] = list_group

In [ ]:
training_csv
#do final pass over the implications and get rid of list format
for i in training_csv.index:
    list_implications.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["implications"].values[0]))))

In [ ]:
training_csv["implications"] = list_implications

## 1) Regular

# 50/50 dataframe

In [ ]:
training_csv

In [ ]:
offY_df = training_csv.where(training_csv["offensiveYN"] == "[OffY]").dropna(subset=['offensiveYN'])
num_offY = len(offY_df.index)
offN_df = training_csv.where(training_csv["offensiveYN"] == "[OffN]").dropna(subset=['offensiveYN']).sample(n=num_offY).reset_index()
fifty_df = pd.concat([offY_df, offN_df])
fifty_df = fifty_df.sample(frac=1).reset_index(drop=True)

In [ ]:
fifty_df

In [ ]:
training_csv = fifty_df

In [ ]:
final_input = []
final_output = []
for i in training_csv.index:
    final_input.append("[boi] " + str(training_csv.loc[[i]]["post"].values[0]) + " [eoi]")
    if (training_csv.loc[[i]]["whoTarget"].values[0] == 0.0 and training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffY]"):
        print("here")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + "[ind] [eoo]")
    elif (training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffN]"):
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoo]")
    else:
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]")

In [ ]:
regular_df = pd.DataFrame()

In [ ]:
regular_df["input"] = final_input
regular_df["output"] = final_output

In [ ]:
regular_df

In [ ]:
regular_df.to_csv("SBIC.trn.2.5050.csv", index=False)

In [ ]:
predictions_df = pd.read_csv("predictions_regular.csv", index_col=0)

In [ ]:
predictions_df

## 2) Full Scramble

In [ ]:
final_input = []
final_output = []
for i in training_csv.index:
    # If it isn't offensive, do the usual (from "regular" scramble)
    if (training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffN]"):
        final_input.append("[boi] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoi]")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoo]")
    # Otherwise, scramble up the input and output according to what is above
    elif (training_csv.loc[[i]]["whoTarget"].values[0] == 0.0 and training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffY]"):
        # Case 1: given offensiveness, group, and stereotype, get post
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + "[ind] [eoi]") 
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoo]")
        # Case 3: given offensiveness, generate post, stereotype, and group
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoi]") 
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls] " + "[ind] [eoo]")
        # Case 4: regular 
        final_input.append("[boi] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoi]")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + "[ind] [eoo]")
    else:
        # Case 1: given offensiveness, group, and stereotype, get post
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoi]")
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoo]")
        # Case 2: given offensiveness and group, generate stereotype and post
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [eoi]")
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]") 
        # Case 3: given offensiveness, generate post, stereotype, and group
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoi]")
        final_output.append("[boo] [bos]" + str(training_csv.loc[[i]]["post"].values[0]) + " [cls] [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]")         
        # Case 4: regular
        final_input.append("[boi] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoi]")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]")

In [ ]:
final_output[0:150]

In [ ]:
full_scramble_df = pd.DataFrame()

In [ ]:
full_scramble_df["input"] = final_input
full_scramble_df["output"] = final_output

In [ ]:
full_scramble_df

In [ ]:
full_scramble_df.to_csv("full.scramble.dev.csv", index=False)

In [ ]:
pd.read_csv("")

# MORE DATA

'acl.id' is the unique ID of the entry.

'Text' is the content which has been entered. All content is synthetic.

'Label' is a binary variable, indicating whether or not the content has been identified as hateful. It takes two values: hate, nothate.

'Type' is a categorical variable, providing a secondary label for hateful content. For hate it can take five values: Animosity, Derogation, Dehumanization, Threatening and Support for Hateful Entities. Please see the paper for more detail. For nothate the 'type' is 'none'. In round 1 the 'type' was not given and is marked as 'notgiven'.

'Target' is a categorical variable, providing the group that is attacked by the hate. It can include intersectional characteristics and multiple groups can be identified. For nothate the type is 'none'. Note that in round 1 the 'target' was not given and is marked as 'notgiven'.

'Level' reports whether the entry is original content or a perturbation.

'Round' is a categorical variable. It gives the round of data entry (1, 2, 3 or 4) with a letter for whether the entry is original content ('a') or a perturbation ('b'). Perturbations were not made for round 1.

'Round.base' is a categorical variable. It gives the round of data entry, indicated with just a number (1, 2, 3 or 4).

'Split' is a categorical variable. it gives the data split that the entry has been assigned to. This can take the values 'train', 'dev' and 'test'. The choice of splits is explained in the paper.

'Annotator' is a categorical variable. It gives the annotator who entered the content. Annotator IDs are random alphanumeric strings. There are 20 annotators in the dataset.

'acl.id.matched' is the ID of the matched entry, connecting the original (given in 'acl.id') and the perturbed version.

For identities (recorded under 'Target') we use shorthand labels to constructed the dataset, which can be converted (and grouped) as follows:

In [ ]:
import pandas as pd
c=pd.read_csv("https://raw.githubusercontent.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.2.csv", index_col=0)
c.drop(["acl.id", "X1", "acl.id.matched"], axis=1)
# get dynahate model into formal to test our model on (predict), calculate acc
# columns for training_csv: post, implications, offy/n, whoTarget, group
# columns to use for DynaHate : text (post), label (offy/n), target (group), (no stereotypes? See how model functions)

In [ ]:
# none but if its offensive
# round 2 and above only 
c = c[(c["round"] != "1")]
c

In [ ]:
c["label"] = c["label"].apply(lambda label: 1 if label=="nothate" else 0)

In [ ]:
c = c[c['target'].map(c['target'].value_counts()) >= 200].reset_index()

In [ ]:
#clean up groupings to be what model is used to from SBIC
c["target"] = c["target"].map({"notgiven" : "notgiven", "wom":"women", "bla":"black", "jew":"jewish", "mus":"muslim", "immig":"immigrants", "asi.south":"south asians", "none" : "none", "transgender" : "transgender people", "ref":"refugee", "arab":"arabic people", "indig" : "indigenous", "mixed.race" : "mixed people", "asi" : "asian", "african" : "african", "hispanic":"hispanic people", "mus.wom" : "muslim women"})

In [ ]:
c["target"].value_counts().to_dict()

In [ ]:
#columns to use for DynaHate : text (post), label (offy/n), target (group), (no stereotypes? See how model functions)
#["post", "group", "implications", "whoTarget", "offensiveYN"]
c.drop(["index", "acl.id", "X1", "round", "acl.id.matched", "annotator", "level"], axis=1, index=None, inplace=True)
c.columns=["post", "offensiveYN", "type", "group", "round.base", "split"]
c

In [ ]:
#split data into dev, test, and train
dev_dyna, test_dyna, train_dyna = [y for x, y in c.groupby('split', as_index=False)]
dev_dyna.drop(["split"], axis=1, inplace=True)
test_dyna.drop(["split"], axis=1, inplace=True)
train_dyna.drop(["split"], axis=1, inplace=True)
# QUESTION: What rows are necessary, and how are they translatable to SBIC?
# if it's too complicated, you can take only the stage3 posts and see how our model does on the posts that we know have implications? 
# And we can measure the offensiveness classification (ie. stage 1) separately

In [ ]:
all_data = pd.DataFrame()
all_data = all_data.append(train_dyna)[train_dyna.columns.tolist()]
all_data = all_data.append(training_csv).dropna(axis=1, how="all")
all_data["whoTarget"] = [1.0] * len(all_data)

In [ ]:
all_data["implications"] = all_data["implications"].fillna("[]")

In [ ]:
#columns to use for DynaHate : text (post), label (offy/n), target (group), (no stereotypes? See how model functions)
#["post", "group", "implications", "whoTarget", "offensiveYN"]

In [ ]:
#merge each file
from functools import reduce
df1 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg1.tsv", sep='\t')
df2 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_SAP_posts.tsv", sep="\t")
df3 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg2.tsv", sep="\t")
df4 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg3.tsv", sep="\t")
df5 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg2_posts.tsv", sep="\t")
df6 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg3_posts.tsv", sep="\t")
df7 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg1_posts.tsv", sep="\t")

# combine each stage together
stg1_df = df1.merge(df7["post"], left_index=True, right_index=True)
#stg2_df = df3.merge(df5["post"], left_index=True, right_index=True)
stg3_df = df4.merge(df6["post"], left_index=True, right_index=True)
total_df = pd.concat([stg1_df, stg2_df, stg3_df], ignore_index=True)
total_df
#data_frames = [df1, df2, df3, df4, df5, df6, df7]
#df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
#                                            how='outer'), data_frames)

In [ ]:
stg1_df.drop(["ID", "class"], axis=1, inplace=True)
stg1_df["offensiveYN"] = ["[OffN]"] * len(stg1_df)
stg1_df["whoTarget"] = [1.0] * len(stg1_df)

In [ ]:
#Add offn from stage one to balance out 
# deduplicate non offensive posts (compared to SBIC) 
stg3_df.drop(["ID"], axis=1, inplace=True)
stg3_df.columns = ["group", "implications", "post"]
stg3_df["offensiveYN"] = ["[OffY]"] * len(stg3_df)
stg3_df["whoTarget"] = [1.0] * len(stg3_df)

In [ ]:
stg3_df = stg3_df.append(stg1_df)

In [ ]:
all_data.append(stg3_df).reset_index()

# Accuracy : Structure

- accuracy of structure (are the tokens in the right order/right place? Good first indicator for models to see if at least the structure is learned correctly

#### for scramble model/regular model
- traverse strings - if next token is not is expected:
    - check if it is an incorrect prediction in the first place (those will not count)
    - if it is not an incorrect prediction, check the count on each token. (we can start with a simple true/false, but: structure is more than just if its in the right order and place, it's about if it makes sense)

In [ ]:
# for script, make this neat
import pandas as pd

In [ ]:
# merge dataframes
preds = pd.read_csv("predictions_2_1000.csv", index_col=0)
preds.columns = ["input", "generated"]

In [ ]:
real_val = pd.read_csv("SBIC.dev.1.csv")

In [ ]:
merged_df = real_val.merge(preds, on="input", how="inner")

In [ ]:
list_of_data_preds = []
list_of_data_real = []
merged_df["output"] = merged_df["output"].apply(lambda x: x.replace("[boi] ", "")).apply(
    lambda x: x.replace(" [eoi]", ""))

merged_df["generated"] = merged_df["generated"].apply(lambda x: x.replace("<pad> ", "")).apply(
    lambda x: x.replace(" <pad>", ""))

y_data_real = merged_df["output"].values
y_data_preds = merged_df["generated"].values

# filter out incorrect results
for i in range(0, len(merged_df)):
    list_of_data_preds.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_preds[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))
    list_of_data_real.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_real[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))

list_of_data_real = pd.DataFrame(list_of_data_real)
list_of_data_preds = pd.DataFrame(list_of_data_preds)
list_of_data_real[0] = list_of_data_real[0].apply(lambda label: 1 if label == "OffN" else 0)
list_of_data_preds[0] = list_of_data_preds[0].apply(lambda label: 1 if label == "OffN" else 0)
mask = (list_of_data_real[0] == list_of_data_preds[0])

In [ ]:
# merge dataframes
preds = pd.read_csv("predictions_3_pt3.csv", index_col=0)
preds.columns = ["input", "generated"]

In [ ]:
preds

In [ ]:
real_val

In [ ]:
real_val = pd.read_csv("SBIC.dev.scramble.3.5050.csv")

In [ ]:
#merged_df = real_val.merge(preds, on="input", how="inner")
merged_df = real_val.merge(preds.drop_duplicates(subset=['input']), how='inner')

In [ ]:
merged_df

In [ ]:
# build up dataframe yourself (don't do this if structure is bad, so check structure first)

In [ ]:
list_of_data_preds = []
list_of_data_real = []
merged_df["output"] = merged_df["output"].apply(lambda x: x.replace("[boi] ", "")).apply(
    lambda x: x.replace(" [eoi]", ""))

merged_df["generated"] = merged_df["generated"].apply(lambda x: x.replace("<pad> ", "")).apply(
    lambda x: x.replace(" <pad>", ""))

y_data_real = merged_df["output"].values
y_data_preds = merged_df["generated"].values

# filter out incorrect results
for i in range(0, len(merged_df)):
    list_of_data_preds.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_preds[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))
    list_of_data_real.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_real[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))

list_of_data_real = pd.DataFrame(list_of_data_real)
list_of_data_preds = pd.DataFrame(list_of_data_preds)

In [ ]:
list_of_data_preds

In [ ]:
list_of_data_real[0] = list_of_data_real[0].apply(lambda label: 1 if label == "OffN" else 0)
list_of_data_preds[0] = list_of_data_preds[0].apply(lambda label: 1 if label == "OffN" else 0)
mask = (list_of_data_real[0] == list_of_data_preds[0])

In [ ]:
import numpy as np
merged_df_preds = merged_df[np.array(mask,dtype=bool)].reset_index()

In [ ]:
merged_df_preds

In [ ]:
merged_df_regular = merged_df[np.array(mask,dtype=bool)].reset_index()

In [ ]:
merged_df_regular

In [ ]:
import re
import numpy as np
total_values = []
# TODO: add in a check to make sure the prediction is correct before adding it to total_values
for i in range(0, len(merged_df_preds)):
    # find all tokens between square brackets
    tokens_input = re.findall("(?<=\[)[^]]+(?=\])", merged_df_preds["output"].tolist()[i])
    tokens_generated = re.findall("(?<=\[)[^]]+(?=\])", merged_df_preds["generated"].tolist()[i])
    #compare the arrays elementwise to determine number of tokens correctly preserved, divide by total amount of tokens preserved
    try:
        total_values.append((np.array(tokens_input)==np.array(tokens_generated)).sum()/len(tokens_input))
    #short term solution to deal with arrays that are not the same length: talk in meeting about best way to mitigate
    except:
        total_values.append(0);
print(np.mean(total_values))

In [23]:
import re
re.findall("(?!<=\[)[^]]+(?=\])", "[boo] this is an example [boi] that would be [eoo] useful")

['[boo', ' this is an example [boi', ' that would be [eoo']

In [ ]:
"\|(?![^\[]*\])"

In [ ]:
accuracy_structure = np.mean(total_values)

In [ ]:
real_val["input"].tolist()[0]

# Regular model

In [60]:
predictions = pd.read_csv("predictions_test_2.csv", index_col=0)
predictions.transpose().to_dict()

{0: {'Source Text': '[boi] There are too many costas in this country [eoi]',
  'Generated Text': '<pad> <pad> <pad> <pad> <pad> linguri<pad> <pad> (<pad> <pad> -<pad> <pad> m<pad> <pad> reteta<pad> <pad> în<pad> <pad> –<pad> <pad> Ach<pad> <pad> zu<pad> <pad>,<pad> <pad> î<pad> <pad> ü<pad> <pad> :<pad> <pad> a<pad> <pad> s<pad> <pad> beauftragte<pad> <pad> ende<pad> <pad> An<pad> <pad> ă<pad> <pad> Diese'},
 1: {'Source Text': '[boi] There is way too many m0ng0l0id kids at my school,  they should be sent to the special school [eoi]',
  'Generated Text': '<pad> <pad> <pad> <pad> <pad> beauftragte<pad> <pad> Stiftung<pad> <pad> zu<pad> <pad> ü<pad> <pad> în<pad> <pad> organiza<pad> <pad> veți<pad> <pad> București<pad> <pad> î<pad> <pad> -<pad> <pad> zuführen<pad> <pad> a<pad> <pad> placut<pad> <pad> linguri<pad> <pad> ă<pad> <pad> Brașov<pad> <pad> ș<pad> <pad> instituții<pad> <pad> schule<pad> <pad> Gemeinde'},
 2: {'Source Text': '[boi] I called my brother a twat and my mom got so mad

In [49]:
pred_test = pd.read_csv("predictions_test.csv", index_col=0)

#for i in range(0, len(pred_test)):
#    print(list(filter(lambda x: x != "", [sentence.strip() for sentence in
#                                                pred_test[i].replace("<pad>", '')])))

In [57]:
pred_test["Generated Text"].values.tolist()[:4]

['<pad> <pad> <pad> <pad> <pad> linguri<pad> <pad> (<pad> <pad> -<pad> <pad> m<pad> <pad> reteta<pad> <pad> în<pad> <pad> –<pad> <pad> Ach<pad> <pad> zu<pad> <pad>,<pad> <pad> î<pad> <pad> ü<pad> <pad> :<pad> <pad> a<pad> <pad> s<pad> <pad> beauftragte<pad> <pad> ende<pad> <pad> An<pad> <pad> ă<pad> <pad> Diese',
 '<pad> <pad> <pad> <pad> <pad> beauftragte<pad> <pad> Stiftung<pad> <pad> zu<pad> <pad> ü<pad> <pad> în<pad> <pad> organiza<pad> <pad> veți<pad> <pad> București<pad> <pad> î<pad> <pad> -<pad> <pad> zuführen<pad> <pad> a<pad> <pad> placut<pad> <pad> linguri<pad> <pad> ă<pad> <pad> Brașov<pad> <pad> ș<pad> <pad> instituții<pad> <pad> schule<pad> <pad> Gemeinde',
 '<pad> <pad> <pad> în<pad> <pad> <pad> <pad> placut<pad> <pad> gasest<pad> <pad> „<pad> <pad> linguri<pad> <pad> („<pad> <pad> între<pad> <pad> Între<pad> <pad> organiza<pad> <pad> ingrijire<pad> <pad> sunteti<pad> <pad> în<pad> <pad>,<pad> <pad> aflat<pad> <pad> -<pad> <pad> î<pad> <pad> zu<pad> <pad> înțeleg<pad> <pa

In [ ]:
# merge dataframes
preds = pd.read_csv("predictions_2_1000.csv", index_col=0)
preds.columns = ["input", "generated"]

In [ ]:
real_val = pd.read_csv("SBIC.dev.1.csv")

In [ ]:
merged_df = real_val.merge(preds, on="input", how="inner")

In [ ]:
import re
import numpy as np
total_values = []
# TODO: add in a check to make sure the prediction is correct before adding it to total_values
for i in range(0, len(merged_df)):
    # find all tokens between square brackets
    tokens_input = re.findall("(?<=\[)[^]]+(?=\])", merged_df["output"].tolist()[i])
    tokens_generated = re.findall("(?<=\[)[^]]+(?=\])", merged_df["generated"].tolist()[i])
    #compare the arrays elementwise to determine number of tokens correctly preserved, divide by total amount of tokens preserved
    try:
        total_values.append((np.array(tokens_input)==np.array(tokens_generated)).sum()/len(tokens_input))
    #short term solution to deal with arrays that are not the same length: talk in meeting about best way to mitigate
    except:
        total_values.append(0);

print(np.mean(total_values))

In [ ]:
accuracy_structure = np.mean(total_values)

In [ ]:
real_val["input"].tolist()[0]

# Accuracy : Generation of Similar values

- frequency of generation (check how frequently the model learns a pattern, generate a heatmap of sorts with keywords)

In [ ]:
len(set(merged_df_preds["generated"].tolist()))/len(set(merged_df_preds["output"]))

In [ ]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
gen_vs_act = []
gen_vs_gen = []
for i in range(0, len(merged_df_preds)):
    generated = merged_df_preds["generated"].tolist()[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    output = merged_df_preds["output"].tolist()[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    if ("[bos]" in generated) and ("[bos]" in output):
        reference = generated.split("[bos]")[1].strip()
        candidate = output.split("[bos]")[1].strip()
        gen = extract_ngrams(reference, 4)
        act = extract_ngrams(candidate, 4)
        # generated vs actual
        gen_vs_act.append(len(set(gen))/len(set(act)))
        # generated unique vs generated total
        gen_vs_gen.append(len(set(gen))/len(gen))

In [ ]:
np.mean(gen_vs_act)

# Accuracy : BLEU scores for the implication score

In [ ]:
!pip install nltk

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
# get data within the implication bounds 
list_preds = merged_df_preds["generated"].tolist()
import re
total_val = []
#split_val = [preds.split("[ste]") for preds in list_preds]
for i in range(0, len(list_preds)):
    list_preds[i] = list_preds[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    real_val_list[i] = real_val_list[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    if "[ste]" in list_preds[i] and "[ste]" in real_val_list[i]:
        reference = [list_preds[i].split("[ste]")[1].strip().split(" ")]
        candidate = real_val_list[i].split("[ste]")[1].strip().split(" ")
        twogram = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        threegram = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        total_val.append((twogram + threegram) / 2)
# 2 and 3 gram mean for all the data 
# compare data to actual implications using BLEU score
# smoothing method1 - NLTK sentence_bleu add
# steps: find where the data that takes implications is in the actual data, compare it with the other one

In [ ]:
np.mean(total_val)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
# get data within the implication bounds 
list_preds = merged_df_regular["generated"].tolist()
import re
total_val = []
#split_val = [preds.split("[ste]") for preds in list_preds]
for i in range(0, len(list_preds)):
    list_preds[i] = list_preds[i].replace("<pad>", "")
    if "[ste]" in list_preds[i] and "[ste]" in real_val_list[i]:
        reference = [list_preds[i].split("[ste]")[1].strip().split(" ")]
        candidate = real_val_list[i].split("[ste]")[1].strip().split(" ")
        twogram = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        threegram = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        total_val.append((twogram + threegram) / 2)
# 2 and 3 gram mean for all the data 
# compare data to actual implications using BLEU score
# smoothing method1 - NLTK sentence_bleu add
# steps: find where the data that takes implications is in the actual data, compare it with the other one

In [ ]:
np.mean(total_val)

# Accuracy of 50/50 model

In [ ]:
preds = pd.read_csv("predictions_2_1000.csv", index_col=0)

In [ ]:
preds["Source Text"] = preds["Source Text"].apply(lambda x: x.replace("[boi] ", "")).apply(lambda x: x.replace(" [eoi]", ""))

In [ ]:
preds["Generated Text"] = preds["Generated Text"].apply(lambda x: x.replace("<pad> ", "")).apply(lambda x: x.replace(" <pad>", ""))
preds.columns = ["post", "output"]

In [ ]:
from sklearn.metrics import f1_score
import re
list_of_data = []
# split on parenthesis
for i in range(0, len(generated_predictions)):
    list_of_data.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                generated_predictions[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))

y_test = pd.read_csv("SBIC.v2.agg.dev.csv", index_col=0)[:1000]
y_test

merged_df = pd.merge(preds, y_test, on="post")
merged_df["offensiveYN"] = merged_df["offensiveYN"].apply(lambda label: 1 if label == 1 else 0)

merged_df["offYNPredictions"] = merged_df["output"].apply(lambda label: 1 if "OffY" in label else 0)

subset_merged_df = merged_df[["output", "targetMinority", "targetCategory", "targetStereotype", "offensiveYN"]]
subset_merged_df[merged_df.output.str.contains('OffY')].head(50)

sum_int = sum(merged_df["offYNPredictions"] == merged_df["offensiveYN"])

accuracy_score = sum_int / 1000

f1_val = f1_score(y_test_off, list_of_data[0])


In [ ]:
accuracy_score

In [ ]:
    
    # Qualitative Testing

    match = []
    for i in range(0, len(list_of_data)):
        if list(y_test["offensiveYN"].values)[i] == 0:
            list_of_data_val = str(list(list_of_data[1].values)[i])
            y_test_val = list(y_test["group"].values)
            match.append(any(list_of_data_val in s for s in y_test_val))
            
    raw_accuracy_group = sum(match) / len(match)


    with open(args.output, 'w') as f:
        f.write("raw accuracy of the Offensive YN: \n" + str(accuracy_score))
        f.write("f1 accuracy score of Offensive YN: \n" + str(f1_val))
        f.write("raw accuracy score of the group tokens: \n" + str(raw_accuracy_group))

    f.close()

In [ ]:
import pandas as pd

In [ ]:
predictions_df = pd.read_csv("predictions_5.csv")
predictions_df.drop(predictions_df.filter(regex="Unname"),axis=1, inplace=True)
predictions_df.columns = ["input", "output"]

In [ ]:
set(predictions_df["output"])

In [ ]:
truth_df = pd.read_csv("SBIC.dev.scramble.4.csv")
merged_df = truth_df.merge(predictions_df, on="input", how="inner")

In [ ]:
merged_df.columns=["input", "output_real", "output_model"]
merged_df

In [ ]:
dict_val = merged_df[["output_real", "output_model"]].transpose().to_dict()

In [ ]:
{k: dict_val[k] for k in list(dict_val)[:10]}

In [ ]:
import pandas as pd
predictions_df = pd.read_csv("predictions_3.csv")

In [8]:
training_csv = "SBIC.trn.scramble.4.csv"
validation_csv = "SBIC.dev.scramble.4.csv"


training_df = pd.read_csv(training_csv)
validation_df = pd.read_csv(validation_csv)

In [11]:
training_df.sample(frac=1.0)[:50].to_csv("50_examples.4.csv")

,input,output
195147,[boi] [bos] little black kid gets his ass kick...,[boo] [OffY] [ste] black kids are fun to watch...
164297,[boi] [bos] What’s the difference between a Je...,"[boo] [OffY] [grp] jewish folks,terrorism vict..."
90825,[boi] [bos] I met a Jewish girl today and she ...,[boo] [OffY] [grp] jewish folks [ste] jews had...
100960,[boi] [bos] Immigrants Welcome (play book)\n1....,[boo] [grp] immigrants [ste] insinuates immig...
15073,[boi] [bos] So i was in America and you know s...,"[boo] [grp] mass shooting victims [ste] [""ame..."
170132,[boi] [bos] Why are all black people fast? All...,[boo] [OffY] [ste] black people run from the p...
22167,[boi] [bos] What did Jesus say before he died?...,"[boo] [grp] christian folks [ste] [""christian..."
45269,[boi] [bos] what do you call a black man with ...,[boo] [OffY] [grp] black folks [eoo]
39516,[boi] [bos] how do I cope knowing\nthat girls ...,"[boo] [OffY] [grp] men [ste] [""is promiscuous""..."
21219,[boi] [bos] We bring people from shithole coun...,"[boo] [OffY] [ste] [""democrats only get votes ..."


In [75]:
def literal_return(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError) as e:
        return []

In [76]:
import ast
i = 0;
original_len = len(dev_csv)
while i < original_len:
    list_val = literal_return(dev_csv["implications"].tolist()[i])
    if (list_val != None and len(list_val) != 0):
        s=dev_csv.iloc[[i],] # pick the row you want to do repeat
        s=s.reindex(s.index.repeat(len(list_val)))# repeat the row by the giving number 
        s.loc[:,'implications']=list_val # change the value
        dev_csv=pd.concat([dev_csv,s]) #append to the original df 
        dev_csv.reset_index(drop=True, inplace=True)
    i = i + 1

#do final pass over the implications and get rid of list format
i = 0;
while i < original_len:
    if "[" in dev_csv["implications"].tolist()[i]:
        dev_csv.drop(index=i, axis=0, inplace=True)
        dev_csv.reset_index()
    i = i + 1

dev_csv.reset_index(drop=True)

,post,group,implications,whoTarget,offensiveYN
0,"Over 10,000 arrests for child trafficking sinc...",[],[],NaN,[OffN]
1,PSA\nY'all should stay away from stupidslutscl...,[],[],0.0,[OffN]
2,PSA: Remember to share the Atomic Blackpill on...,[],[],0.0,[OffN]
3,Partly Cloudy today! With a high of 16C.and a ...,[],[],NaN,[OffN]
4,Party bus going to LA live at this bar/bowling...,[],[],NaN,[OffN]
...,...,...,...,...,...
10181,♪ &#9835; ♬ believe it or not i 'm gassing the...,"[""jewish folks""]",died in the holocaust.,1.0,[OffY]
10182,♪ &#9835; ♬ believe it or not i 'm gassing the...,"[""jewish folks""]",have a troubled past,1.0,[OffY]
10183,♪ &#9835; ♬ believe it or not i 'm gassing the...,"[""jewish folks""]",jews deserved to be burned,1.0,[OffY]
10184,♪ &#9835; ♬ believe it or not i 'm gassing the...,"[""jewish folks""]",making a joke about the holocaust.,1.0,[OffY]


In [77]:
list_group = []
for i in range(0, len(dev_csv)):
    list_group.append(",".join(map(str, literal_return(dev_csv.iloc[[i]]["group"].values[0]))))
    #list_group.append(",".join(map(str, literal_return(dev_csv.iloc[[i]]["group"]) if not None)))
    
dev_csv["group"] = list_group
dev_csv.reset_index(drop=True)

,post,group,implications,whoTarget,offensiveYN
0,"Over 10,000 arrests for child trafficking sinc...",,[],NaN,[OffN]
1,PSA\nY'all should stay away from stupidslutscl...,,[],0.0,[OffN]
2,PSA: Remember to share the Atomic Blackpill on...,,[],0.0,[OffN]
3,Partly Cloudy today! With a high of 16C.and a ...,,[],NaN,[OffN]
4,Party bus going to LA live at this bar/bowling...,,[],NaN,[OffN]
...,...,...,...,...,...
10181,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,died in the holocaust.,1.0,[OffY]
10182,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,have a troubled past,1.0,[OffY]
10183,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,jews deserved to be burned,1.0,[OffY]
10184,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,making a joke about the holocaust.,1.0,[OffY]


In [97]:
training_csv.columns

Index(['post', 'group', 'implications', 'whoTarget', 'offensiveYN'], dtype='object')

In [78]:
import ast
i = 0;
original_len = len(training_csv)
while i < original_len:
    list_val = literal_return(training_csv["implications"].tolist()[i])
    if (list_val != None and len(list_val) != 0):
        s=training_csv.iloc[[i],] # pick the row you want to do repeat
        s=s.reindex(s.index.repeat(len(list_val)))# repeat the row by the giving number 
        s.loc[:,'implications']=list_val # change the value
        training_csv=pd.concat([training_csv,s]) #append to the original df 
        training_csv.reset_index(drop=True, inplace=True)
    i = i + 1

#do final pass over the implications and get rid of list format
i = 0;
while i < original_len:
    if "[" in training_csv["implications"].tolist()[i]:
        training_csv.drop(index=i, axis=0, inplace=True)
        training_csv.reset_index()
    i = i + 1

training_csv.reset_index(drop=True)

,post,group,implications,whoTarget,offensiveYN
0,Nelly got all the hoes singin lol,[],[],0.0,[OffN]
1,Nelson snaps me a pic of him in a cocoon sleep...,[],[],0.0,[OffN]
2,Neocels\n..is the term to describe normies who...,[],[],0.0,[OffN]
3,Nerd followers: I'm monitoring hashtag/mention...,[],[],NaN,[OffN]
4,Net neutrality is pretty irrelevant when a han...,[],[],0.0,[OffN]
...,...,...,...,...,...
58655,⁠Why do wh*te people like to play hockey? It’...,"[""black folks"", ""white folks""]",makes light of racial tension with police,1.0,[OffY]
58656,🏢🏢 👳🏽‍♂️✈️➡️➡️🏢🏢➡️➡️💥💥➡️➡️🤷🏻‍♂️🤷🏻‍♂️\n\n,"[""muslim folks""]",muslims are terrorists,1.0,[OffY]
58657,🏢🏢 👳🏽‍♂️✈️➡️➡️🏢🏢➡️➡️💥💥➡️➡️🤷🏻‍♂️🤷🏻‍♂️\n\n,"[""muslim folks""]",muslims like bombs,1.0,[OffY]
58658,🚨BREAKING: illegal alien 5x deported on 7 felo...,"[""immigrants""]",immigrants are criminals,1.0,[OffY]


In [79]:
list_group = []
for i in range(0, len(training_csv)):
    list_group.append(",".join(map(str, literal_return(training_csv.iloc[[i]]["group"].values[0]))))
    #list_group.append(",".join(map(str, literal_return(training_csv.iloc[[i]]["group"]) if not None)))
    
training_csv["group"] = list_group
training_csv.reset_index(drop=True)

,post,group,implications,whoTarget,offensiveYN
0,Nelly got all the hoes singin lol,,[],0.0,[OffN]
1,Nelson snaps me a pic of him in a cocoon sleep...,,[],0.0,[OffN]
2,Neocels\n..is the term to describe normies who...,,[],0.0,[OffN]
3,Nerd followers: I'm monitoring hashtag/mention...,,[],NaN,[OffN]
4,Net neutrality is pretty irrelevant when a han...,,[],0.0,[OffN]
...,...,...,...,...,...
58655,⁠Why do wh*te people like to play hockey? It’...,"black folks,white folks",makes light of racial tension with police,1.0,[OffY]
58656,🏢🏢 👳🏽‍♂️✈️➡️➡️🏢🏢➡️➡️💥💥➡️➡️🤷🏻‍♂️🤷🏻‍♂️\n\n,muslim folks,muslims are terrorists,1.0,[OffY]
58657,🏢🏢 👳🏽‍♂️✈️➡️➡️🏢🏢➡️➡️💥💥➡️➡️🤷🏻‍♂️🤷🏻‍♂️\n\n,muslim folks,muslims like bombs,1.0,[OffY]
58658,🚨BREAKING: illegal alien 5x deported on 7 felo...,immigrants,immigrants are criminals,1.0,[OffY]


In [80]:
training_csv

,post,group,implications,whoTarget,offensiveYN
17712,Nelly got all the hoes singin lol,,[],0.0,[OffN]
17713,Nelson snaps me a pic of him in a cocoon sleep...,,[],0.0,[OffN]
17714,Neocels\n..is the term to describe normies who...,,[],0.0,[OffN]
17715,Nerd followers: I'm monitoring hashtag/mention...,,[],NaN,[OffN]
17716,Net neutrality is pretty irrelevant when a han...,,[],0.0,[OffN]
...,...,...,...,...,...
76367,⁠Why do wh*te people like to play hockey? It’...,"black folks,white folks",makes light of racial tension with police,1.0,[OffY]
76368,🏢🏢 👳🏽‍♂️✈️➡️➡️🏢🏢➡️➡️💥💥➡️➡️🤷🏻‍♂️🤷🏻‍♂️\n\n,muslim folks,muslims are terrorists,1.0,[OffY]
76369,🏢🏢 👳🏽‍♂️✈️➡️➡️🏢🏢➡️➡️💥💥➡️➡️🤷🏻‍♂️🤷🏻‍♂️\n\n,muslim folks,muslims like bombs,1.0,[OffY]
76370,🚨BREAKING: illegal alien 5x deported on 7 felo...,immigrants,immigrants are criminals,1.0,[OffY]


In [81]:
training_csv[["implications"]].dropna()[2000:2100]

,implications
19712,"[""fat folks are animals/hippos"", ""physically d..."
19713,[]
19714,[]
19715,[]
19716,[]
...,...
19807,[]
19808,[]
19809,"[""are niggers"", ""black people are dumb"", ""blac..."
19810,"[""ugly people shouldn't be taken seriously"", ""..."


In [13]:
dev_csv

,post,group,implications,whoTarget,offensiveYN
2333,"Over 10,000 arrests for child trafficking sinc...",,[],NaN,[OffN]
2334,PSA\nY'all should stay away from stupidslutscl...,,[],0.0,[OffN]
2335,PSA: Remember to share the Atomic Blackpill on...,,[],0.0,[OffN]
2336,Partly Cloudy today! With a high of 16C.and a ...,,[],NaN,[OffN]
2337,Party bus going to LA live at this bar/bowling...,,[],NaN,[OffN]
...,...,...,...,...,...
12514,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,died in the holocaust.,1.0,[OffY]
12515,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,have a troubled past,1.0,[OffY]
12516,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,jews deserved to be burned,1.0,[OffY]
12517,♪ &#9835; ♬ believe it or not i 'm gassing the...,jewish folks,making a joke about the holocaust.,1.0,[OffY]
